In [ ]:
import pandas as pd
from sqlalchemy import create_engine
SERVER = 'DESKTOP-0NGHB9E'
DATABASE = 'COMERCIO_DW'
USERNAME = 'sa'
PASSWORD = '123456'
DRIVER = "ODBC Driver 17 for SQL Server"
connection_string = f"mssql+pyodbc://{USERNAME}:{PASSWORD}@{SERVER}/{DATABASE}?driver={DRIVER.replace(' ', '+')}"
engine = create_engine(connection_string)
print(" Conexão estabelecida com sucesso!")
tabelas = pd.read_sql("""
    SELECT TABLE_NAME
    FROM INFORMATION_SCHEMA.TABLES
    WHERE TABLE_TYPE = 'BASE TABLE'
""", engine)
print(" Tabelas disponíveis no COMERCIO_DW:")
print(tabelas)
tabelas_dim = [
    "fact.FATO_VENDAS",
    "dim.DIM_CLIENTE",
    "dim.DIM_PRODUTO",
    "dim.DIM_TEMPO",
    "dim.DIM_VENDEDOR",
    "dim.DIM_FORNECEDOR"
]
dfs = {}
for tabela in tabelas_dim:
    try:
        df = pd.read_sql(f"SELECT * FROM {tabela}", engine)
        dfs[tabela] = df
        print(f" {tabela} carregada com {len(df)} registros e {len(df.columns)} colunas")
    except Exception as e:
        print(f"️ Erro ao carregar {tabela}: {e}")
df_vendas = dfs.get("fact.FATO_VENDAS")
if df_vendas is not None:
    display(df_vendas.head())

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
SERVER = 'DESKTOP-0NGHB9E'
DATABASE = 'COMERCIO_DW'
USERNAME = 'sa'
PASSWORD = '123456'
DRIVER = "ODBC Driver 17 for SQL Server"
connection_string = f"mssql+pyodbc://{USERNAME}:{PASSWORD}@{SERVER}/{DATABASE}?driver={DRIVER.replace(' ', '+')}"
engine = create_engine(connection_string)
print(" Conexão estabelecida com sucesso!")
query_pk = """
SELECT
    tc.TABLE_SCHEMA,
    tc.TABLE_NAME,
    kcu.COLUMN_NAME
FROM INFORMATION_SCHEMA.TABLE_CONSTRAINTS tc
JOIN INFORMATION_SCHEMA.KEY_COLUMN_USAGE kcu
    ON tc.CONSTRAINT_NAME = kcu.CONSTRAINT_NAME
WHERE tc.CONSTRAINT_TYPE = 'PRIMARY KEY'
ORDER BY tc.TABLE_SCHEMA, tc.TABLE_NAME;
"""
df_pk = pd.read_sql(query_pk, engine)
print("\n PRIMARY KEYS ENCONTRADAS:")
display(df_pk)
query_fk = """
SELECT
    fk.name AS FK_Name,
    tp.name AS ParentTable,
    ref.name AS ReferencedTable,
    c1.name AS ParentColumn,
    c2.name AS ReferencedColumn
FROM sys.foreign_keys fk
INNER JOIN sys.foreign_key_columns fkc
    ON fkc.constraint_object_id = fk.object_id
INNER JOIN sys.tables tp
    ON fkc.parent_object_id = tp.object_id
INNER JOIN sys.columns c1
    ON fkc.parent_object_id = c1.object_id AND fkc.parent_column_id = c1.column_id
INNER JOIN sys.tables ref
    ON fkc.referenced_object_id = ref.object_id
INNER JOIN sys.columns c2
    ON fkc.referenced_object_id = c2.object_id AND fkc.referenced_column_id = c2.column_id
ORDER BY tp.name, ref.name;
"""
df_fk = pd.read_sql(query_fk, engine)
print("\n FOREIGN KEYS ENCONTRADAS:")
display(df_fk)

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
SERVER = 'DESKTOP-0NGHB9E'
DATABASE = 'COMERCIO_DW'
USERNAME = 'sa'
PASSWORD = '123456'
DRIVER = "ODBC Driver 17 for SQL Server"
connection_string = f"mssql+pyodbc://{USERNAME}:{PASSWORD}@{SERVER}/{DATABASE}?driver={DRIVER.replace(' ', '+')}"
engine = create_engine(connection_string)
print(" Conexão estabelecida com sucesso!")
campos_criticos = [
    "FATO_ID",
    "DATA_SK",
    "CLIENTE_SK",
    "PRODUTO_SK",
    "VENDEDOR_SK",
    "QTDE",
    "VALOR_UNITARIO",
    "VL_TOTAL"
]
resultados = []
for campo in campos_criticos:
    query = f"SELECT COUNT(*) AS qtd_nulos FROM fact.FATO_VENDAS WHERE {campo} IS NULL"
    qtd_nulos = pd.read_sql(query, engine)['qtd_nulos'][0]
    resultados.append({"Campo": campo, "Nulos": qtd_nulos})
    if qtd_nulos > 0:
        print(f"️ {campo}: {qtd_nulos} valores nulos encontrados")
    else:
        print(f" {campo}: sem valores nulos")
df_nulos = pd.DataFrame(resultados)
print("\n Resumo final de nulos em fact.FATO_VENDAS:")
display(df_nulos)

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
import re
from datetime import datetime
SERVER = 'DESKTOP-0NGHB9E'
DATABASE = 'COMERCIO_DW'
USERNAME = 'sa'
PASSWORD = '123456'
DRIVER = "ODBC Driver 17 for SQL Server"
connection_string = f"mssql+pyodbc://{USERNAME}:{PASSWORD}@{SERVER}/{DATABASE}?driver={DRIVER.replace(' ', '+')}"
engine = create_engine(connection_string)
print(" Conexão estabelecida com sucesso!")
tipos_esperados = {
    "FATO_ID": "int",
    "DATA_SK": "int",
    "CLIENTE_SK": "int",
    "PRODUTO_SK": "int",
    "VENDEDOR_SK": "int",
    "QTDE": "numeric",
    "VALOR_UNITARIO": "numeric",
    "VL_TOTAL": "numeric"
}
print(" Verificando tipos de dados na tabela fact.FATO_VENDAS...\n")
query_tipos = """
SELECT
    COLUMN_NAME,
    DATA_TYPE,
    CHARACTER_MAXIMUM_LENGTH,
    NUMERIC_PRECISION,
    NUMERIC_SCALE
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_SCHEMA = 'fact'
AND TABLE_NAME = 'FATO_VENDAS'
ORDER BY ORDINAL_POSITION
"""
df_tipos_sql = pd.read_sql(query_tipos, engine)
print(" Tipos de dados definidos no SQL Server:")
print(df_tipos_sql)
print("\n" + "="*60 + "\n")
resultados_tipos = []
for campo, tipo_esperado in tipos_esperados.items():
    print(f" Analisando campo: {campo}")
    if campo not in df_tipos_sql['COLUMN_NAME'].values:
        print(f" {campo}: Coluna não encontrada na tabela")
        resultados_tipos.append({
            "Campo": campo,
            "Tipo_Esperado": tipo_esperado,
            "Tipo_SQL": "N/A",
            "Status": "COLUNA_NAO_ENCONTRADA",
            "Inconsistencias": 0
        })
        continue
    tipo_sql = df_tipos_sql[df_tipos_sql['COLUMN_NAME'] == campo]['DATA_TYPE'].iloc[0]
    inconsistencias = 0
    status = "OK"
    if tipo_esperado == "int":
        query_int = f"""
        SELECT COUNT(*) as inconsistencias
        FROM fact.FATO_VENDAS
        WHERE {campo} IS NOT NULL
        AND (
            ISNUMERIC(CAST({campo} AS VARCHAR)) = 0
            OR CAST({campo} AS VARCHAR) LIKE '%.%'
            OR CAST({campo} AS VARCHAR) LIKE '%,%'
        )
        """
        try:
            inconsistencias = pd.read_sql(query_int, engine)['inconsistencias'][0]
        except:
            inconsistencias = 0
    elif tipo_esperado == "numeric":
        query_numeric = f"""
        SELECT COUNT(*) as inconsistencias
        FROM fact.FATO_VENDAS
        WHERE {campo} IS NOT NULL
        AND ISNUMERIC(CAST({campo} AS VARCHAR)) = 0
        """
        try:
            inconsistencias = pd.read_sql(query_numeric, engine)['inconsistencias'][0]
        except:
            inconsistencias = 0
    elif tipo_esperado == "date":
        query_date = f"""
        SELECT COUNT(*) as inconsistencias
        FROM fact.FATO_VENDAS
        WHERE {campo} IS NOT NULL
        AND ISDATE({campo}) = 0
        """
        try:
            inconsistencias = pd.read_sql(query_date, engine)['inconsistencias'][0]
        except:
            inconsistencias = 0
    if inconsistencias > 0:
        status = "INCONSISTENTE"
        print(f"️ {campo}: {inconsistencias} valores com tipo inconsistente")
    else:
        print(f" {campo}: tipos consistentes")
    resultados_tipos.append({
        "Campo": campo,
        "Tipo_Esperado": tipo_esperado,
        "Tipo_SQL": tipo_sql,
        "Status": status,
        "Inconsistencias": inconsistencias
    })
print("\n" + "="*60)
print("\n Verificações adicionais de consistência...\n")
query_data_sk = """
SELECT COUNT(*) as inconsistencias
FROM fact.FATO_VENDAS
WHERE DATA_SK IS NOT NULL
AND (
    LEN(CAST(DATA_SK AS VARCHAR)) != 8
    OR DATA_SK < 19000101
    OR DATA_SK > 21001231
    OR ISDATE(
        SUBSTRING(CAST(DATA_SK AS VARCHAR), 1, 4) + '-' +
        SUBSTRING(CAST(DATA_SK AS VARCHAR), 5, 2) + '-' +
        SUBSTRING(CAST(DATA_SK AS VARCHAR), 7, 2)
    ) = 0
)
"""
try:
    data_sk_inconsistencias = pd.read_sql(query_data_sk, engine)['inconsistencias'][0]
    if data_sk_inconsistencias > 0:
        print(f"️ DATA_SK: {data_sk_inconsistencias} valores com formato de data inválido")
    else:
        print(" DATA_SK: formato de data consistente")
except Exception as e:
    print(f" Erro ao verificar DATA_SK: {e}")
campos_positivos = ["QTDE", "VALOR_UNITARIO", "VL_TOTAL"]
for campo in campos_positivos:
    query_negativo = f"""
    SELECT COUNT(*) as negativos
    FROM fact.FATO_VENDAS
    WHERE {campo} < 0
    """
    try:
        negativos = pd.read_sql(query_negativo, engine)['negativos'][0]
        if negativos > 0:
            print(f"️ {campo}: {negativos} valores negativos encontrados")
        else:
            print(f" {campo}: sem valores negativos")
    except Exception as e:
        print(f" Erro ao verificar {campo}: {e}")
df_tipos = pd.DataFrame(resultados_tipos)
print("\n Resumo final de consistência de tipos em fact.FATO_VENDAS:")
print(df_tipos)
df_problemas = df_tipos[df_tipos['Status'] != 'OK']
if not df_problemas.empty:
    print("\n Campos com inconsistências de tipo:")
    print(df_problemas)
else:
    print("\n Todos os campos estão com tipos consistentes!")
engine.dispose()
print("\n Conexão fechada.")

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
SERVER = 'DESKTOP-0NGHB9E'
DATABASE = 'COMERCIO_DW'
USERNAME = 'sa'
PASSWORD = '123456'
DRIVER = "ODBC Driver 17 for SQL Server"
connection_string = f"mssql+pyodbc://{USERNAME}:{PASSWORD}@{SERVER}/{DATABASE}?driver={DRIVER.replace(' ', '+')}"
engine = create_engine(connection_string)
print(" Conexão estabelecida com sucesso!")
dimensoes_config = {
    "dim.DIM_CLIENTE": {
        "chave_sk": "CLIENTE_SK",
        "chave_fato": "CLIENTE_SK",
        "nome_dimensao": "Cliente"
    },
    "dim.DIM_PRODUTO": {
        "chave_sk": "PRODUTO_SK",
        "chave_fato": "PRODUTO_SK",
        "nome_dimensao": "Produto"
    },
    "dim.DIM_VENDEDOR": {
        "chave_sk": "VENDEDOR_SK",
        "chave_fato": "VENDEDOR_SK",
        "nome_dimensao": "Vendedor"
    },
    "dim.DIM_TEMPO": {
        "chave_sk": "DATA_SK",
        "chave_fato": "DATA_SK",
        "nome_dimensao": "Data"
    }
}
tabela_fato = "fact.FATO_VENDAS"
print(" Verificando registros órfãos nas dimensões...\n")
resultados_orfaos = []
for tabela_dim, config in dimensoes_config.items():
    print(f" Analisando dimensão: {config['nome_dimensao']} ({tabela_dim})")
    chave_sk = config['chave_sk']
    chave_fato = config['chave_fato']
    nome_dimensao = config['nome_dimensao']
    try:
        query_existe_dim = f"""
        SELECT COUNT(*) as existe
        FROM INFORMATION_SCHEMA.TABLES
        WHERE TABLE_SCHEMA + '.' + TABLE_NAME = '{tabela_dim.replace('dim.', 'dim')}'
        """
        existe_dim = pd.read_sql(query_existe_dim, engine)['existe'][0]
        if existe_dim == 0:
            print(f" {nome_dimensao}: Tabela {tabela_dim} não encontrada")
            resultados_orfaos.append({
                "Dimensao": nome_dimensao,
                "Tabela": tabela_dim,
                "Total_Registros": 0,
                "Registros_Orfaos": 0,
                "Percentual_Orfaos": 0,
                "Status": "TABELA_NAO_ENCONTRADA"
            })
            continue
        query_total = f"SELECT COUNT(*) as total FROM {tabela_dim}"
        total_registros = pd.read_sql(query_total, engine)['total'][0]
        query_orfaos = f"""
        SELECT COUNT(*) as orfaos
        FROM {tabela_dim} d
        WHERE NOT EXISTS (
            SELECT 1
            FROM {tabela_fato} f
            WHERE f.{chave_fato} = d.{chave_sk}
        )
        AND d.{chave_sk} IS NOT NULL
        """
        registros_orfaos = pd.read_sql(query_orfaos, engine)['orfaos'][0]
        percentual_orfaos = (registros_orfaos / total_registros * 100) if total_registros > 0 else 0
        if registros_orfaos == 0:
            status = "OK"
            print(f" {nome_dimensao}: Sem registros órfãos")
        elif percentual_orfaos <= 5:
            status = "ATENÇÃO"
            print(f"️ {nome_dimensao}: {registros_orfaos} registros órfãos ({percentual_orfaos:.1f}%)")
        else:
            status = "CRÍTICO"
            print(f" {nome_dimensao}: {registros_orfaos} registros órfãos ({percentual_orfaos:.1f}%)")
        resultados_orfaos.append({
            "Dimensao": nome_dimensao,
            "Tabela": tabela_dim,
            "Total_Registros": total_registros,
            "Registros_Orfaos": registros_orfaos,
            "Percentual_Orfaos": round(percentual_orfaos, 2),
            "Status": status
        })
    except Exception as e:
        print(f" Erro ao analisar {nome_dimensao}: {e}")
        resultados_orfaos.append({
            "Dimensao": nome_dimensao,
            "Tabela": tabela_dim,
            "Total_Registros": 0,
            "Registros_Orfaos": 0,
            "Percentual_Orfaos": 0,
            "Status": "ERRO"
        })
print("\n" + "="*80)
print("\n Verificando chaves na fato sem correspondência nas dimensões...\n")
resultados_sem_dim = []
for tabela_dim, config in dimensoes_config.items():
    print(f" Analisando chave: {config['chave_fato']} -> {config['nome_dimensao']}")
    chave_sk = config['chave_sk']
    chave_fato = config['chave_fato']
    nome_dimensao = config['nome_dimensao']
    try:
        query_sem_dim = f"""
        SELECT COUNT(*) as sem_dimensao
        FROM {tabela_fato} f
        WHERE f.{chave_fato} IS NOT NULL
        AND NOT EXISTS (
            SELECT 1
            FROM {tabela_dim} d
            WHERE d.{chave_sk} = f.{chave_fato}
        )
        """
        sem_dimensao = pd.read_sql(query_sem_dim, engine)['sem_dimensao'][0]
        query_total_fato = f"""
        SELECT COUNT(*) as total
        FROM {tabela_fato}
        WHERE {chave_fato} IS NOT NULL
        """
        total_fato = pd.read_sql(query_total_fato, engine)['total'][0]
        percentual_sem_dim = (sem_dimensao / total_fato * 100) if total_fato > 0 else 0
        if sem_dimensao == 0:
            status = "OK"
            print(f" {nome_dimensao}: Todas as chaves da fato têm correspondência")
        elif percentual_sem_dim <= 1:
            status = "ATENÇÃO"
            print(f"️ {nome_dimensao}: {sem_dimensao} registros na fato sem dimensão ({percentual_sem_dim:.1f}%)")
        else:
            status = "CRÍTICO"
            print(f" {nome_dimensao}: {sem_dimensao} registros na fato sem dimensão ({percentual_sem_dim:.1f}%)")
        resultados_sem_dim.append({
            "Dimensao": nome_dimensao,
            "Chave_Fato": chave_fato,
            "Total_Fato": total_fato,
            "Sem_Dimensao": sem_dimensao,
            "Percentual_Sem_Dim": round(percentual_sem_dim, 2),
            "Status": status
        })
    except Exception as e:
        print(f" Erro ao analisar chave {chave_fato}: {e}")
        resultados_sem_dim.append({
            "Dimensao": nome_dimensao,
            "Chave_Fato": chave_fato,
            "Total_Fato": 0,
            "Sem_Dimensao": 0,
            "Percentual_Sem_Dim": 0,
            "Status": "ERRO"
        })
print("\n" + "="*80)
print("\n Detalhando registros órfãos (primeiros 10 de cada dimensão)...\n")
for tabela_dim, config in dimensoes_config.items():
    nome_dimensao = config['nome_dimensao']
    chave_sk = config['chave_sk']
    chave_fato = config['chave_fato']
    query_detalhes = f"""
    SELECT TOP 10 d.{chave_sk}, d.*
    FROM {tabela_dim} d
    WHERE NOT EXISTS (
        SELECT 1
        FROM {tabela_fato} f
        WHERE f.{chave_fato} = d.{chave_sk}
    )
    AND d.{chave_sk} IS NOT NULL
    ORDER BY d.{chave_sk}
    """
    try:
        df_detalhes = pd.read_sql(query_detalhes, engine)
        if not df_detalhes.empty:
            print(f" Primeiros registros órfãos em {nome_dimensao}:")
            print(df_detalhes.head())
            print()
    except Exception as e:
        print(f" Erro ao detalhar {nome_dimensao}: {e}")
df_orfaos = pd.DataFrame(resultados_orfaos)
df_sem_dim = pd.DataFrame(resultados_sem_dim)
print("\n Resumo final - Registros órfãos nas dimensões:")
print(df_orfaos)
print("\n Resumo final - Registros na fato sem dimensão:")
print(df_sem_dim)
df_problemas_orfaos = df_orfaos[df_orfaos['Status'].isin(['CRÍTICO', 'ATENÇÃO'])]
df_problemas_sem_dim = df_sem_dim[df_sem_dim['Status'].isin(['CRÍTICO', 'ATENÇÃO'])]
if not df_problemas_orfaos.empty:
    print("\n Dimensões com registros órfãos:")
    print(df_problemas_orfaos[['Dimensao', 'Registros_Orfaos', 'Percentual_Orfaos', 'Status']])
if not df_problemas_sem_dim.empty:
    print("\n Chaves na fato sem correspondência nas dimensões:")
    print(df_problemas_sem_dim[['Dimensao', 'Sem_Dimensao', 'Percentual_Sem_Dim', 'Status']])
if df_problemas_orfaos.empty and df_problemas_sem_dim.empty:
    print("\n Todas as dimensões estão consistentes com a tabela fato!")
engine.dispose()
print("\n Conexão fechada.")

In [ ]:
import pandas as pd

In [ ]:
import numpy as np
from sqlalchemy import create_engine

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns
from datetime import datetime, timedelta

In [ ]:
import warnings
warnings.filterwarnings('ignore')
SERVER = 'DESKTOP-0NGHB9E'
DATABASE = 'COMERCIO_DW'
USERNAME = 'sa'
PASSWORD = '123456'
DRIVER = "ODBC Driver 17 for SQL Server"
connection_string = f"mssql+pyodbc://{USERNAME}:{PASSWORD}@{SERVER}/{DATABASE}?driver={DRIVER.replace(' ', '+')}"
engine = create_engine(connection_string)
print(" Conexão estabelecida com sucesso!")
print("="*60)
print(" Extraindo dados da tabela fato...")
query_principal = """
SELECT
    f.FATO_ID,
    f.DATA_SK,
    f.CLIENTE_SK,
    f.PRODUTO_SK,
    f.VENDEDOR_SK,
    f.QTDE,
    f.VALOR_UNITARIO,
    f.VL_TOTAL,
    f.COD_VENDA_ORIGEM,
   -- Dados da dimensão cliente
    c.COD_CLIENTE,
    c.NOME            AS NOME_CLIENTE,
    c.CPF,
    c.EMAIL,
    c.DT_NASCIMENTO,
    c.ATIVO           AS CLIENTE_ATIVO,
    -- Dados da dimensão produto
    p.PRODUTO_SK,
    p.COD_PRODUTO,
    p.DESCRICAO,
    p.CATEGORIA,
    p.PRECO_ATUAL,
    p.DT_INICIO,
    p.DT_FIM,
    p.ATIVO           AS PRODUTO_ATIVO,
    p.HASH_ATTR,
    -- Dados da dimensão vendedor
    v.VENDEDOR_SK,
    v.COD_VENDEDOR,
    v.NOME           AS NOME_VENDEDOR,
    v.REGIAO as REGIAO_VENDEDOR,
    v.DT_ADMISSAO,
    v.DT_INICIO,
    v.DT_FIM,
    v.ATIVO          AS VENDEDOR_ATIVO,
    v.HASH_ATTR,
    -- Dados da dimensão TEMPO
    t.DATA_SK,
    t.DATA_COMPLETA,
    t.ANO,
    t.MES,
    t.TRIMESTRE,
    t.NOME_MES,
    t.DIA,
    t.NOME_DIA,
    t.FIM_DE_SEMANA
FROM fact.FATO_VENDAS f
LEFT JOIN dim.DIM_CLIENTE c ON f.CLIENTE_SK = c.CLIENTE_SK
LEFT JOIN dim.DIM_PRODUTO p ON f.PRODUTO_SK = p.PRODUTO_SK
LEFT JOIN dim.DIM_VENDEDOR v ON f.VENDEDOR_SK = v.VENDEDOR_SK
LEFT JOIN dim.DIM_TEMPO t ON f.DATA_SK = t.DATA_SK
WHERE f.VL_TOTAL IS NOT NULL
"""
df = pd.read_sql(query_principal, engine)
df = df.loc[:, ~df.columns.duplicated()]
print(f" Dados extraídos: {len(df):,} registros")
print(f" Colunas disponíveis: {len(df.columns)}")
print("\n Primeiras linhas dos dados:")
print(df.head())
print("\n" + "="*60)
print(" ANÁLISE EXPLORATÓRIA DOS DADOS")
print("="*60)
print("\n Estatísticas descritivas das vendas:")
print(df[['QTDE', 'VALOR_UNITARIO', 'VL_TOTAL']].describe())
print(f"\n Período dos dados: {df['ANO'].min()} a {df['ANO'].max()}")
print(f" Categorias de produtos: {df['CATEGORIA'].nunique()}")
print(f" Clientes únicos: {df['CLIENTE_SK'].nunique()}")
print(f"‍ Vendedores únicos: {df['VENDEDOR_SK'].nunique()}")
print("\n" + "="*60)
print(" CRIANDO COLUNAS DERIVADAS")
print("="*60)
print("\n1️⃣ Criando faixas de venda baseadas em percentis...")
p25 = df['VL_TOTAL'].quantile(0.25)
p75 = df['VL_TOTAL'].quantile(0.75)
print(f"    Percentil 25%: R$ {p25:,.2f}")
print(f"    Percentil 75%: R$ {p75:,.2f}")
def classificar_venda(valor):
    if valor <= p25:
        return 'Low'
    elif valor <= p75:
        return 'Medium'
    else:
        return 'High'
df['FAIXA_VENDA'] = df['VL_TOTAL'].apply(classificar_venda)
print("    Coluna FAIXA_VENDA criada!")
print(f"    Distribuição das faixas:")
print(df['FAIXA_VENDA'].value_counts())
print("\n2️⃣ Criando indicador de sazonalidade...")
def classificar_sazonalidade(mes):
    if mes in [12, 1, 2]:
        return 'Verão'
    elif mes in [3, 4, 5]:
        return 'Outono'
    elif mes in [6, 7, 8]:
        return 'Inverno'
    else:
        return 'Primavera'
df['SAZONALIDADE'] = df['MES'].apply(classificar_sazonalidade)
print("    Coluna SAZONALIDADE criada!")
print(f"    Distribuição por estação:")
print(df['SAZONALIDADE'].value_counts())
print("\n3️⃣ Criando classificação de performance do vendedor...")
vendas_por_vendedor = df.groupby('VENDEDOR_SK')['VL_TOTAL'].sum().reset_index()
vendas_por_vendedor.columns = ['VENDEDOR_SK', 'TOTAL_VENDAS_VENDEDOR']
p33_vendedor = vendas_por_vendedor['TOTAL_VENDAS_VENDEDOR'].quantile(0.33)
p66_vendedor = vendas_por_vendedor['TOTAL_VENDAS_VENDEDOR'].quantile(0.66)
print(f"    Percentil 33% vendedores: R$ {p33_vendedor:,.2f}")
print(f"    Percentil 66% vendedores: R$ {p66_vendedor:,.2f}")
def classificar_performance_vendedor(total_vendas):
    if total_vendas <= p33_vendedor:
        return 'Baixa Performance'
    elif total_vendas <= p66_vendedor:
        return 'Média Performance'
    else:
        return 'Alta Performance'
vendas_por_vendedor['PERFORMANCE_VENDEDOR'] = vendas_por_vendedor['TOTAL_VENDAS_VENDEDOR'].apply(classificar_performance_vendedor)
df = df.merge(vendas_por_vendedor[['VENDEDOR_SK', 'TOTAL_VENDAS_VENDEDOR', 'PERFORMANCE_VENDEDOR']],
              on='VENDEDOR_SK', how='left')
print("    Coluna PERFORMANCE_VENDEDOR criada!")
print(f"    Distribuição de performance:")
print(df['PERFORMANCE_VENDEDOR'].value_counts())
print("\n4️⃣ Criando classificação de ticket médio...")
df['TICKET_MEDIO'] = df['VL_TOTAL'] / df['QTDE']
p50_ticket = df['TICKET_MEDIO'].quantile(0.50)
def classificar_ticket(ticket):
    if ticket <= p50_ticket:
        return 'Ticket Baixo'
    else:
        return 'Ticket Alto'
df['CLASSIFICACAO_TICKET'] = df['TICKET_MEDIO'].apply(classificar_ticket)
print("    Colunas TICKET_MEDIO e CLASSIFICACAO_TICKET criadas!")
print(f"    Ticket médio mediano: R$ {p50_ticket:.2f}")
print(f"    Distribuição de tickets:")
print(df['CLASSIFICACAO_TICKET'].value_counts())
print("\n5️⃣ Criando categoria de cliente...")
vendas_por_cliente = df.groupby('CLIENTE_SK')['VL_TOTAL'].sum().reset_index()
vendas_por_cliente.columns = ['CLIENTE_SK', 'TOTAL_VENDAS_CLIENTE']
p20_cliente = vendas_por_cliente['TOTAL_VENDAS_CLIENTE'].quantile(0.20)
p80_cliente = vendas_por_cliente['TOTAL_VENDAS_CLIENTE'].quantile(0.80)
def classificar_cliente(total_vendas):
    if total_vendas <= p20_cliente:
        return 'Bronze'
    elif total_vendas <= p80_cliente:
        return 'Prata'
    else:
        return 'Ouro'
vendas_por_cliente['CATEGORIA_CLIENTE'] = vendas_por_cliente['TOTAL_VENDAS_CLIENTE'].apply(classificar_cliente)
df = df.merge(vendas_por_cliente[['CLIENTE_SK', 'TOTAL_VENDAS_CLIENTE', 'CATEGORIA_CLIENTE']],
              on='CLIENTE_SK', how='left')
print("    Coluna CATEGORIA_CLIENTE criada!")
print(f"    Distribuição de categorias:")
print(df['CATEGORIA_CLIENTE'].value_counts())
print("\n6️⃣ Criando indicador de dia útil...")
def classificar_dia_util(dia_semana):
    if dia_semana in [1, 7]:
        return 'Fim de Semana'
    else:
        return 'Dia Útil'
df['TIPO_DIA'] = df['FIM_DE_SEMANA'].apply(classificar_dia_util)
print("    Coluna TIPO_DIA criada!")
print(f"    Distribuição por tipo de dia:")
print(df['TIPO_DIA'].value_counts())
print("\n" + "="*60)
print(" RESUMO DAS COLUNAS DERIVADAS CRIADAS")
print("="*60)
colunas_derivadas = [
    'FAIXA_VENDA', 'SAZONALIDADE', 'PERFORMANCE_VENDEDOR',
    'TICKET_MEDIO', 'CLASSIFICACAO_TICKET', 'CATEGORIA_CLIENTE', 'TIPO_DIA'
]
print(f" Total de colunas derivadas: {len(colunas_derivadas)}")
for i, coluna in enumerate(colunas_derivadas, 1):
    print(f"{i}. {coluna}")
print(f"\n Dataset final: {len(df)} registros x {len(df.columns)} colunas")
print("\n" + "="*60)
print(" ANÁLISES COM AS COLUNAS DERIVADAS")
print("="*60)
print("\n1️⃣ Análise por Faixa de Venda:")
analise_faixa = df.groupby('FAIXA_VENDA').agg({
    'VL_TOTAL': ['count', 'sum', 'mean'],
    'QTDE': 'sum'
}).round(2)
print(analise_faixa)
print("\n2️⃣ Vendas por Sazonalidade:")
analise_sazon = df.groupby('SAZONALIDADE').agg({
    'VL_TOTAL': ['count', 'sum', 'mean']
}).round(2)
print(analise_sazon)
print("\n3️⃣ Análise de Performance dos Vendedores:")
analise_vendedor = df.groupby('PERFORMANCE_VENDEDOR').agg({
    'VL_TOTAL': ['count', 'sum', 'mean'],
    'VENDEDOR_SK': 'nunique'
}).round(2)
print(analise_vendedor)
print("\n4️⃣ Análise por Categoria de Cliente:")
analise_cliente = df.groupby('CATEGORIA_CLIENTE').agg({
    'VL_TOTAL': ['count', 'sum', 'mean'],
    'CLIENTE_SK': 'nunique'
}).round(2)
print(analise_cliente)
print("\n" + "="*60)
print(" EXPORTANDO DADOS TRANSFORMADOS")
print("="*60)
df.to_csv('dados_transformados_comercio_dw.csv', index=False, encoding='utf-8-sig')
print(" Dados exportados para 'dados_transformados_comercio_dw.csv'")
df_derivadas = df[['FATO_ID'] + colunas_derivadas]
df_derivadas.to_csv('colunas_derivadas_comercio_dw.csv', index=False, encoding='utf-8-sig')
print(" Colunas derivadas exportadas para 'colunas_derivadas_comercio_dw.csv'")
print(f"\n RESUMO FINAL:")
print(f"   • Registros processados: {len(df):,}")
print(f"   • Colunas originais: {len(df.columns) - len(colunas_derivadas)}")
print(f"   • Colunas derivadas criadas: {len(colunas_derivadas)}")
print(f"   • Total de colunas: {len(df.columns)}")
engine.dispose()
print("\n Conexão com banco fechada.")
print(" Processo de transformação concluído com sucesso!")

In [ ]:
import pandas as pd

In [ ]:
import numpy as np
print("\n7️⃣ Criando coluna Novo vs Recorrente por cliente...")
df['DATA_COMPLETA'] = pd.to_datetime(df['DATA_COMPLETA'])
primeira_compra_cliente = df.groupby('CLIENTE_SK')['DATA_COMPLETA'].min().reset_index()
primeira_compra_cliente.columns = ['CLIENTE_SK', 'PRIMEIRA_COMPRA']
df = df.merge(primeira_compra_cliente, on='CLIENTE_SK', how='left')
df['NOVO_RECORRENTE'] = np.where(df['DATA_COMPLETA'] == df['PRIMEIRA_COMPRA'],
                                 'Novo',
                                 'Recorrente')
print("    Coluna NOVO_RECORRENTE criada!")
print(f"    Distribuição:")
print(df['NOVO_RECORRENTE'].value_counts())
print("\n" + "="*60)
print(" CRIANDO DATAFRAMES PRÉ-AGRUPADOS")
print("="*60)
df_vendas_mes_produto = (
    df.groupby(['ANO', 'MES', 'COD_PRODUTO', 'DESCRICAO'])
      .agg(
          VENDAS_TOTAIS=('VL_TOTAL', 'sum'),
          QTD_TOTAL=('QTDE', 'sum'),
          TICKET_MEDIO=('VL_TOTAL', 'mean')
      )
      .reset_index()
      .sort_values(['ANO', 'MES'])
)
print("\n DataFrame 1: Vendas por Mês e Produto")
print(df_vendas_mes_produto.head())
df_vendas_regiao_categoria = (
    df.groupby(['REGIAO_VENDEDOR', 'CATEGORIA'])
      .agg(
          VENDAS_TOTAIS=('VL_TOTAL', 'sum'),
          QTD_TOTAL=('QTDE', 'sum'),
          PRODUTOS_DISTINTOS=('COD_PRODUTO', 'nunique'),
          VENDEDORES_ATIVOS=('VENDEDOR_SK', 'nunique')
      )
      .reset_index()
      .sort_values(['REGIAO_VENDEDOR', 'CATEGORIA'])
)
print("\n DataFrame 2: Vendas por Região e Categoria de Produto")
print(df_vendas_regiao_categoria.head())
print("\n EXPORTANDO DATASET PRÉ-AGRUPADO...")
df_vendas_mes_produto.to_csv("vendas_por_mes_produto.csv", index=False, encoding="utf-8-sig")
print(" Arquivo 'vendas_por_mes_produto.csv' exportado com sucesso!")